In [1]:
from bs4 import BeautifulSoup as bs 
import pandas as pd   
import numpy as np
import os

In [2]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [3]:
def get_soup(text):
    return bs(text, 'html.parser')

In [4]:
def has_notice(soup):
    return soup.find(class_="notice") is not None

In [5]:
def fetch_value(soup, feature, class_name):
    additions = ['مُكيّف', 'إغلاق مركزي', 'جهاز إنذار', 'مسجل CD', 'فتحة سقف', 'جنطات مغنيسيوم', 'فرش جلد', 'وسادة حماية هوائية']
    value = np.NaN
    
    if class_name == "driving-table":
        if feature == 'الموديل':
            value = soup.find(class_=class_name).find("h3").text
        elif feature == 'موديل سنة':
            h5_list = soup.find(class_=class_name).findAll("h5")
            for item in h5_list:
                if 'موديل سنة' in item.text:
                    value = int(item.text.split("موديل سنة")[1])

    elif class_name == "post-price":
        value = soup.find(class_=class_name).text.split("\n")[0].split(' ')[0]
    else:
        if feature == "سعر التأمين":
            value = int(soup.findAll(class_="list_ads")[1].text.split('\n')[19])
        else:
            features_list = soup.find(class_=class_name).text.split("\n")
            if feature in additions:
                value = 1 if feature in features_list else 0
            else:
                for match in features_list:
                    if feature in match:
                        value = match.split(feature)[1]

    return value

In [6]:
def process_files(directory_path):
    files = os.listdir(directory_path)
    data_list = []

    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        text = read_file(file_path)
        soup = get_soup(text)

        if not has_notice(soup):
            current_car_info = {
                "السعر": float(fetch_value(soup, "السعر", "post-price")),
                "الموديل": fetch_value(soup, "الموديل", "driving-table"),
                'موديل سنة': fetch_value(soup, 'موديل سنة', "driving-table"),
                "لون السيارة": fetch_value(soup, "لون السيارة", "list_ads"),
                'قوة الماتور': float(fetch_value(soup, 'قوة الماتور', "list_ads")),
                "عدد الركاب": fetch_value(soup, "عدد الركاب", "list_ads"),
                "عداد السيارة": fetch_value(soup, "عداد السيارة", "list_ads"),
                "أصحاب سابقون": fetch_value(soup, "أصحاب سابقون", "list_ads"),
                'أصل السيارة': fetch_value(soup, 'أصل السيارة', "list_ads"),
                'رخصة السيارة': fetch_value(soup, 'رخصة السيارة', "list_ads"),
                "نوع الوقود": fetch_value(soup, "نوع الوقود", "list_ads"),
                'نوع الجير': fetch_value(soup, 'نوع الجير', "list_ads"),
                'الزجاج': fetch_value(soup, 'الزجاج', "list_ads"),
                "سعر التأمين": fetch_value(soup, "سعر التأمين", "list_ads"),
                'وسادة حماية هوائية': fetch_value(soup, 'وسادة حماية هوائية', "list_ads"),
                'فرش جلد': fetch_value(soup, 'فرش جلد', "list_ads"),
                'جنطات مغنيسيوم': fetch_value(soup, 'جنطات مغنيسيوم', "list_ads"),
                'فتحة سقف': fetch_value(soup, 'فتحة سقف', "list_ads"),
                'مسجل CD': fetch_value(soup, 'مسجل CD', "list_ads"),
                'إغلاق مركزي': fetch_value(soup, 'إغلاق مركزي', "list_ads"),
                'مُكيّف': fetch_value(soup, 'مُكيّف', "list_ads"),
                'جهاز إنذار': fetch_value(soup, 'جهاز إنذار', "list_ads"),
            }
            data_list.append(current_car_info)

    return pd.DataFrame(data_list)

In [7]:
directory_path = r'C:\\Users\\HP\\Downloads\\data'
cars_data = process_files(directory_path)
cars_data.to_csv('Data.csv', index=False)

In [8]:
data=pd.read_csv('C:\\Users\\HP\\Downloads\\Data.csv')

In [9]:
data

,السعر,الموديل,موديل سنة,لون السيارة,قوة الماتور,عدد الركاب,عداد السيارة,أصحاب سابقون,أصل السيارة,رخصة السيارة,...,الزجاج,سعر التأمين,وسادة حماية هوائية,فرش جلد,جنطات مغنيسيوم,فتحة سقف,مسجل CD,إغلاق مركزي,مُكيّف,جهاز إنذار
0,100000.0,كيا اوبتيما,2014,أبيض عاجي,2000.0,4+1,75000,يد اولى,خصوصي,فلسطينية,...,الكتروني,3090,1,1,1,1,1,1,1,1
1,60000.0,كيا سورينتو,2007,سكني,2500.0,7+1,130000,2,خصوصي,فلسطينية,...,الكتروني,2740,1,1,1,1,1,1,1,1
2,43500.0,هونداي افانتي,2006,سكني,1600.0,NaN,NaN,NaN,خصوصي,فلسطينية,...,الكتروني,2340,1,1,1,0,1,1,1,1
3,5500.0,فيات 127,1982,بيج,906.0,4+1,شغال,00,خصوصي,فلسطينية,...,يدوي,967,0,0,0,0,1,0,1,0
4,54000.0,بيجو 208,2014,فضي,1200.0,4+1,38000,NaN,خصوصي,فلسطينية,...,الكتروني,2035,1,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,45000.0,كيا مورننغ,2015,فيراني,1000.0,4+1,130000,2,خصوصي,فلسطينية,...,الكتروني,1935,1,0,1,0,0,1,1,1
6860,48000.0,فورد ترانزيت,2002,أبيض,2400.0,7+1,00000,5,عمومي,فلسطينية,...,الكتروني,1345,0,0,0,0,0,0,0,0
6861,87000.0,بيجو بارتنر,2018,فضي,1600.0,4+1,50000,يد صفر,خصوصي,فلسطينية,...,الكتروني,2863,1,0,0,0,1,1,1,1
6862,126000.0,كيا سورينتو,2017,بترولي,2200.0,6+1,100,ثانيه,خصوصي,فلسطينية,...,الكتروني,3895,1,1,1,1,1,1,1,1
